In [ ]:
import copy
import numpy as np
from scipy.linalg import norm
from scipy import linalg 
import math 



#### Решение методом Гаусса

Входные параметры:

In [ ]:
eps = 0.0001
n=10
b = [[1/i]  for i in range(1,n)]
a = [[0] * (n) for i in range(n) ]
for i in range(n):
    for j in range(n):
        if i == j:
            a[i][j] = 1
        else:
            a[i][j] = round(1/(i+j+2), 10)
#for row in a:
#    print(' '.join([str(elem) for elem in row]))
#print('\n')
b

In [2]:
def gaussFunc(a):
    eps = 1e-16

    c = numpy.array(a)
    a = numpy.array(a)

    len1 = len(a[:, 0])
    len2 = len(a[0, :])
    vectB = copy.deepcopy(a[:, len1])

    for g in range(len1):

        max = abs(a[g][g])
        my = g
        t1 = g
        while t1 < len1:
            if abs(a[t1][g]) > max:
                max = abs(a[t1][g])
                my = t1
            t1 += 1


        if my != g:
            b = copy.deepcopy(a[g])
            a[g] = copy.deepcopy(a[my])
            a[my] = copy.deepcopy(b)

        amain = float(a[g][g])

        z = g
        while z < len2:
            a[g][z] = a[g][z] / amain
            z += 1

        j = g + 1

        while j < len1:
            b = a[j][g]
            z = g

            while z < len2:
                a[j][z] = a[j][z] - a[g][z] * b
                z += 1
            j += 1

    a = backTrace(a, len1, len2)


    return a

def vectorN(c, a, len1, vectB): 
    c = numpy.array(c)
    a = numpy.array(a)
    vectB = numpy.array(vectB)

    b = numpy.zeros((len1))

    i = 0


    while i<len1:
        j = 0
        while j<len1:

            b[i]+=c[i][j]*a[j]

            j+=1

        i=i+1
    c = copy.deepcopy(b)
    print("!")

    for i in range(len1):
        c[i] = abs(c[i] - vectB[i])

    return c

In [3]:
import numpy
n = 10
a = [[0] * (n + 1) for i in range(n)]
for i in range(n):
    for j in range(n + 1):
        for k in range(n):
            a[k][n] = 1/(k+1)  
        if i == j:
            a[i][j] = 1
        else:
            a[i][j] = 1/(i+j+1)
b=numpy.array(a)
q=gaussFunc(b)            
#for row in a:
#    print(' '.join([str(elem) for elem in row]))
print('Решение методом Гаусса')
print('\n Ответ:', q)

NameError: name 'copy' is not defined

#### Решение методом Зейделя

In [ ]:
def seidel(A, b, eps):
    k = 0
    n = len(A)
    x = [.0 for i in range(n)]
    converge = False
    while not converge:
        x_new = np.copy(x)
        for i in range(n):
            k = k + 1
            s1 = sum(A[i][j] * x_new[j] for j in range(i))
            s2 = sum(A[i][j] * x[j] for j in range(i + 1, n))
            x_new[i] = (b[i] - s1 - s2) / A[i][i]
            
        print('x',k,'=', x_new)
        print('x',k+1,'=',x)
        converge = sqrt(sum((x_new[i] - x[i]) ** 2 for i in range(n))) <= eps
        x = x_new
    print(k)
    return x


Итерационный процесс вида $x^{(k+1)}=x^{(k)}B+c$ сходится, когда $\left\|B\right\|\lt1$. Проверим это

In [ ]:
def norm1(u):
    r = numpy.zeros(n + 1)
    j = 0
    while j < n:
        i = 0
        while i < n:
            r[j] += abs(u[i][j])
            i += 1
        j = j + 1
    return (max(r)) 

In [ ]:
#матрица итерационной системы
u = [[0] * (n) for i in range(n)]
for i in range(n):
    for j in range(n):
        if i == j:
            u[i][j] = 0
        else:
            u[i][j] = -round(a[i][j]/a[i][i],2)

#норма матрицы итерационной системы               


Следовательно условие сходимости невыполнено 

In [ ]:
A = np.array(a)
f = np.array(b)
x = seidel(a, b, 0.0001)



In [ ]:
#Собственные значения матрицы 
D, V=numpy.linalg.eig(a)
print(D)
print('lambda_max',max(D))
print('lambda_min',min(D)) 
    
#Число обусловленности     
r1=numpy.zeros(n+1)
j=0
while j<(n):
    i = 0
    while i<(n):

        r1[i]=r1[i]+a[i][j]

        i=i+1
    j=j+1
#print(r1)
print('\n Норма матрицы A=',max(r1))
w=numpy.linalg.inv(a)

s=numpy.zeros(n)
while i<(n): 
    while j<(n):
        s[i][j]+=(a[i][j] * w[j][i])
        j+=1
    i+=1
    
r2=numpy.zeros(n+1)
j=0
while j<(n):
    i = 0
    while i<(n):

        r2[j]+=abs(w[i][j])

        i=i+1
    j=j+1
print('Норма матрицы A^-1=',max(r2))
print('m1,2=', max(r1)*max(r2))
    
    
print('\n') 
#print(q)

i=0
x = numpy.zeros((n))
while i<(n):
        j = 0
        while j<(n):
            x[i]+=a[i][j]*q[j]  
            j+=1
        i=i+1
   
print('\n Невязка')         
print('Ax=',x)
i=0
while i<(n):
    x[i]=abs(x[i]-1/(i+1))  
    i+=1

f=numpy.zeros(n+1)
i=0
while i<(n+1):
    f[i]=1/(i+1)
    i+=1
print('\nf =', f) 

print('\n')        
print('Ax-f =',x)

print('\n')



Итерационный метод Зейделя $u^{(k+1)}=-(L+D)^{-1}Uu^{(k)}+(L+D)^{-1}f$

In [ ]:

LD = [[0] * (n) for i in range(n) ]
for i in range(n):
    for j in range(n):
        if i == j:
            LD[i][j] = 1
        elif i > j:
            LD[i][j] = round(1/(i+j+2),2)
        else:
            LD[i][j] = 0

U = [[0] * (n) for i in range(n) ]
for i in range(n):
    for j in range(n):
        if i < j:
            U[i][j] = round(1/(i+j+2), 10)
        else:
            U[i][j] = 0

R = -np.dot(linalg.inv(LD), U)
F = np.dot(linalg.inv(LD), b)